In [1]:
# Install required libraries
!pip install transformers accelerate sentencepiece gradio requests

# Import necessary modules
import torch
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

##############################################
# 1. Load the Falcon-7B-Instruct Model
##############################################
model_name = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)



##############################################
# 3. Create the Chat Prompt with Personalized & Updated Events
##############################################
def create_chat_prompt(user_input: str, user_profile: dict) -> str:
    """
    Build a prompt that includes:
    - A system instruction explaining the bot's role.
    - The user's profile (city and interests).
    - A summary of up-to-date event data from Google.
    - The user's question.
    """
    city = user_profile.get("city", "")
    interests = user_profile.get("interests", [])

    # Retrieve up-to-date events using the Google API
    events = get_updated_events(city, interests)
    if events:
        event_summaries = []
        for event in events:
            summary = (
                f"Title: {event['title']}\n"
                f"Link: {event['link']}\n"
                f"Description: {event['snippet']}\n"
                "----\n"
            )
            event_summaries.append(summary)
        events_info = "\n".join(event_summaries)
    else:
        events_info = "No updated events found matching your preferences."

    # Define the system prompt for the chatbot
    system_prompt = (
        "You are an event advisor chatbot. Use the following updated event data from Google to answer the user's question "
        "in a personalized and helpful manner.\n\n"
        "User Preferences:\n"
        f"City: {city}\n"
        f"Interests: {', '.join(interests)}\n\n"
        "Event Data:\n"
        f"{events_info}\n"
        "End of event data.\n\n"
        "Respond concisely and helpfully."
    )

    # Combine the system prompt with the user's question
    combined_prompt = f"{system_prompt}\n\nUser: {user_input}\nBot:"
    return combined_prompt



##############################################
# 5. Create a Chatbot Interface Using Gradio
##############################################
def chatbot_interface(user_input: str, city: str, interests_str: str) -> str:
    """
    Gradio interface function.
    - user_input: The user's question.
    - city: The user's city for event personalization.
    - interests_str: Comma-separated string of the user's interests.
    """
    # Convert the interests string into a list (removing extra spaces)
    interests = [interest.strip() for interest in interests_str.split(",") if interest.strip()]
    user_profile = {
         "city": city,
         "interests": interests
    }
    # Build the combined prompt with dynamic event data
    prompt = create_chat_prompt(user_input, user_profile)
    # Generate the chatbot response
    response = generate_response(prompt)
    return response

# Launch the Gradio interface for interactive testing
iface = gr.Interface(
    fn=chatbot_interface,
    inputs=[
         gr.Textbox(lines=2, label="Your Question", placeholder="Ask about events..."),
         gr.Textbox(lines=1, label="City", placeholder="e.g., New York"),
         gr.Textbox(lines=1, label="Interests", placeholder="e.g., technology, startups")
    ],
    outputs="text",
    title="Real-Time Personalized Event Advisor Chatbot"
)

iface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b923e0c32bce4b54fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
